<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/PCB_Notebook1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import confusion_matrix
import warnings

warnings.filterwarnings("ignore")

/home/shouvik/Desktop/DeepLearning/dl/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/shouvik/Desktop/DeepLearning/dl/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/shouvik/Desktop/DeepLearning/dl/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/shouvik/De

In [2]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [3]:
# X = np.load('drive/My Drive/Copy of xtrain.npy')
# y = np.load('drive/My Drive/Copy of ytrain.npy')
X_train = pickle.load(open(f'X_train298.pkl', 'rb'))
y_train = pickle.load(open(f'y_train298.pkl', 'rb'))
y_test = pickle.load(open(f'y_test298.pkl', 'rb'))
X_test = pickle.load(open(f'X_test298.pkl', 'rb'))

In [4]:
X_test.shape

torch.Size([60, 3, 300, 300])

In [5]:
# X = torch.from_numpy(X)
# X = X.permute(0,3,1,2)
# y = torch.from_numpy(y)
# X.shape, y.shape

In [6]:
# pickle.dump(X, open("augX.pkl", 'wb'), protocol=4)
# pickle.dump(resized_imageset, open(f'resized_imageset1.pkl', 'wb'))
# X = pickle.load(open(f'resized_imageset1.pkl', 'rb'))

In [7]:
# X_train = pickle.load(open(f'xtrain298.pkl', 'rb'))
# y_train = pickle.load(open(f'ytrain298.pkl', 'rb'))
# X_test = pickle.load(open(f'xtest298.pkl', 'rb'))
# y_test = pickle.load(open(f'ytest298.pkl', 'rb'))

In [8]:
# def splitTrainTest(X,y):
#   shuffled_indices = torch.randperm(X.shape[0])
#   ul = math.floor(0.8*X.shape[0])
#   train_indices = shuffled_indices[:ul]
#   test_indices = shuffled_indices[ul:]
#   # train_indices.shape[0] + test_indices.shape[0]
#   X_train = X[train_indices]
#   y_train = y[train_indices]  
#   X_test = X[test_indices]
#   y_test = y[test_indices]
#   print('y_train -> [0]:{} [1]:{}'.format((y_train == 0).sum().item(), (y_train == 1).sum().item()))
#   print('y_test -> [0]:{} [1]:{}'.format((y_test == 0).sum().item(), (y_test == 1).sum().item()))
#   return X_train, y_train, X_test, y_test

In [9]:
# X_train, y_train, X_test, y_test = splitTrainTest(X, y)

In [10]:
# pickle.dump(X_train, open("X_train298.pkl", 'wb'), protocol=4)
# pickle.dump(X_test, open("X_test298.pkl", 'wb'), protocol=4)
# pickle.dump(y_train, open("y_train298.pkl", 'wb'), protocol=4)
# pickle.dump(y_test, open("y_test298.pkl", 'wb'), protocol=4)

Normalizing


In [11]:
def getNormalized(X,ns):
  flattened_channels = X.reshape(3,-1)
  channel_mean = flattened_channels.mean(dim = 1)
  channel_stddev = flattened_channels.std(dim = 1)
  preprocess2 = transforms.Compose([
                      transforms.Normalize(channel_mean, channel_stddev)
  ])


  temptwo = torch.tensor([])
  for i in range(X.shape[0]):
    a = preprocess2(X[i])
    temptwo = torch.cat([temptwo, a.reshape(1,3,ns,ns)])
  
  return temptwo


Resizing

In [12]:
def imageSetResize(newSize,X):
  preprocess1 = transforms.Compose([
                        transforms.ToPILImage(),
                        transforms.Resize(newSize),
                        transforms.ToTensor()])
  
  temp = torch.tensor([])
  for i in range(X.shape[0]):
    a = preprocess1(X[i])
    temp = torch.cat([temp, a.reshape(1,3,newSize,newSize)])

  return temp 

In [13]:
imsize = 256
X_train = imageSetResize(imsize, X_train.float())
X_train = getNormalized(X_train.float(),imsize)
X_test = imageSetResize(imsize, X_test.float())
X_test = getNormalized(X_test.float(),imsize)

In [14]:
X_train.shape

torch.Size([238, 3, 256, 256])

Checking Distribution

In [15]:
(y_test == 1.).sum(), (y_test == 0.).sum(), (y_train == 0.).sum(), (y_train == 1.).sum()

(tensor(30), tensor(30), tensor(119), tensor(119))

Alexnet

In [16]:
anet = models.alexnet(pretrained = True)

In [17]:
def giveAlexnetRepresentations(X):
  anet_representations = torch.tensor([])

  for i in range(X.shape[0]):
    anet_representations = torch.cat([anet_representations, anet(X[i].unsqueeze(0))])

  return anet_representations 

In [18]:
model1 = nn.Sequential(
    nn.Linear(1000, 300),
    nn.Dropout(p=0.3),
    nn.ReLU(),
    nn.Linear(300,100),
    nn.Dropout(p=0.6),
    nn.ReLU(),
    nn.Linear(100,2),
    nn.Dropout(p=0.5),
    nn.ReLU(),
    nn.Softmax(dim=1)
)

Operations

Shuffling and Batching

In [19]:
def labelize(p):
  labelized_preds = []
  for i in p:
    l = 0. if i[0]>i[1] else 1.
    labelized_preds.append(l)

  return torch.tensor(labelized_preds)

In [20]:
def shuffle_and_batch(X,y,num,bs):
  shuffled_indices = torch.randperm(X.shape[0])
  newX = X[shuffled_indices]
  newY = y[shuffled_indices]

  X_batches = []
  y_batches = []
  for i in range(num):
    X_batches.append(X[i*bs:(i+1)*bs])
    y_batches.append(y_train[i*bs:(i+1)*bs])

  return X_batches, y_batches

In [21]:
X_train_representations = giveAlexnetRepresentations(X_train)
X_test_representations = giveAlexnetRepresentations(X_test.float())

In [22]:
X_train_representations.shape, X_test_representations.shape 

(torch.Size([238, 1000]), torch.Size([60, 1000]))

Training and Testing

In [23]:
# device = torch.device('cpu')
# model1 = model1.to(device)
# tb = SummaryWriter()
prev_testacc = -float('inf')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(lr = 1e-3, params = model1.parameters())
model_path = './salexnet.pth'
prev_testacc = -float('inf')


for epoch in range(10):
    X_batches, y_batches = shuffle_and_batch(X_train_representations, y_train, 4, 67)
  # X_batches = X_batches.to(device)
  # y_batches = y_batches.to(device)
    print('Epoch {}'.format(epoch))
    for i in range(1):#len(X_batches)):
        pred = model1(X_batches[i])
        loss = criterion(pred,y_batches[i].long())
        optimizer.zero_grad()
        loss.backward(retain_graph = True)
        optimizer.step()


  # Checking model on training set
    train_preds = model1(X_train_representations)
    train_loss = criterion(train_preds, y_train.long())
    train_preds = labelize(train_preds)
    train_prediction_comparisons = (y_train == train_preds)
    train_accuracy = float(train_prediction_comparisons.sum())/float(y_train.shape[0])
    print('Training Loss:{}\tTraining Accuracy:{}'.format(train_loss.item(), train_accuracy), end='  ')
  
  # Checking model on testing set
    test_preds = model1(X_test_representations)
    test_loss = criterion(test_preds, y_test.long())
    test_preds = labelize(test_preds)
    test_prediction_comparisons = (y_test == test_preds)
    test_accuracy = float(test_prediction_comparisons.sum())/float(y_test.shape[0])
    print('Testing Loss:{}\tTesting Accuracy:{}'.format(test_loss.item(), test_accuracy))
    if test_accuracy < prev_testacc and prev_testacc>0.7:
        break
#     state_dict = model1.state_dict()
    torch.save(model1, model_path)
    prev_testacc = test_accuracy


#     tb.add_scalar('TrainLoss',train_loss, epoch)
#     tb.add_scalar('TestLoss',test_loss, epoch)
#     tb.add_scalar('TrainAccuracy', train_accuracy, epoch)
#     tb.add_scalar('TestAccuracy', test_accuracy, epoch)


Epoch 0
Training Loss:0.6936129927635193	Training Accuracy:0.5126050420168067  Testing Loss:0.691265881061554	Testing Accuracy:0.5
Epoch 1
Training Loss:0.6949495077133179	Training Accuracy:0.49159663865546216  Testing Loss:0.6906247735023499	Testing Accuracy:0.5333333333333333
Epoch 2
Training Loss:0.7011478543281555	Training Accuracy:0.4957983193277311  Testing Loss:0.6885553598403931	Testing Accuracy:0.5166666666666667
Epoch 3
Training Loss:0.6903575658798218	Training Accuracy:0.5126050420168067  Testing Loss:0.6934581995010376	Testing Accuracy:0.5166666666666667
Epoch 4
Training Loss:0.6917905211448669	Training Accuracy:0.5168067226890757  Testing Loss:0.6945813894271851	Testing Accuracy:0.48333333333333334
Epoch 5
Training Loss:0.691349446773529	Training Accuracy:0.5210084033613446  Testing Loss:0.7025631666183472	Testing Accuracy:0.48333333333333334
Epoch 6
Training Loss:0.6939675211906433	Training Accuracy:0.5042016806722689  Testing Loss:0.6931471228599548	Testing Accuracy:0.5


In [24]:
confusion_matrix(y_train, train_preds)

array([[  1, 118],
       [  0, 119]])

In [25]:
confusion_matrix(y_test, test_preds)

array([[ 0, 30],
       [ 0, 30]])

AugSet

In [26]:
# Xaug = pickle.load(open(f'X5040_Alexnet.pkl', 'rb'))
# yaug = pickle.load(open(f'y5040.pkl', 'rb'))

In [27]:
# aug_preds = model1(Xaug)
# aug_loss = criterion(aug_preds, yaug.long())
# aug_preds = labelize(aug_preds)
# aug_prediction_comparisons = (yaug == aug_preds)
# aug_accuracy = float(aug_prediction_comparisons.sum())/float(yaug.shape[0])
# print('AugAccuracy:{}'.format(aug_accuracy), end='  ') 

In [28]:
# confusion_matrix(yaug, aug_preds)

Raw

In [29]:
Xraw = pickle.load(open(f'XAlexnet2669_raw_256.pkl', 'rb'))
yraw = pickle.load(open(f'yAlexnet2669_raw_256.pkl', 'rb'))

In [30]:
a1 = torch.load(model_path)

In [31]:
# raw_preds = model11(Xraw)
# # raw_loss = criterion(raw_preds, labelToOneHot(yraw))
# raw_preds = labelize(raw_preds)
# raw_prediction_comparisons = (yraw == raw_preds)
# raw_accuracy = float(raw_prediction_comparisons.sum())/float(yraw.shape[0])
# print('RawAccuracy:{}'.format(raw_accuracy), end='  ')  

In [32]:
# confusion_matrix(yraw, raw_preds)

In [33]:
# from sklearn.metrics import f1_score
# f1_score(yraw, raw_preds, average = 'weighted')

In [36]:
test_preds = a1(X_test_representations)
test_loss = criterion(test_preds, y_test.long())
#     test_loss = criterion(test_preds,labelToOneHot(y_test))
test_preds = labelize(test_preds)
test_prediction_comparisons = (y_test == test_preds)
test_accuracy = float(test_prediction_comparisons.sum())/float(y_test.shape[0])
print('TestLoss:{} TestAccuracy:{}'.format(test_loss.item(), test_accuracy))
    

TestLoss:0.6931471228599548 TestAccuracy:0.5


Densenet

In [ ]:
# dnet = models.densenet121(pretrained = True)

In [ ]:
# normalized_imageset = getNormalized(X.float())
# resized_imageset = imageSetResize(256, normalized_imageset)

In [ ]:
# def getD121Representations(X):
#   global dnet
#   device = torch.device('cuda:0')
#   dnet_representations = torch.tensor([]).float().to(device)
#   dnet = dnet.to(device)

#   X = X.float().to(device)
#   for i in range(X.shape[0]):
#     dnet_representations = torch.cat([dnet_representations, dnet(X[i].unsqueeze(0))])

#   device = torch.device('cpu')
#   dnet_representations = dnet_representations.to(device)
#   X = X.to(device)
#   dnet = dnet.to(device)
#   return dnet_representations 

In [ ]:
# X_train_representations = getD121Representations(X_train)
# X_test_representations = getD121Representations(X_test)

In [ ]:
# device = torch.device('cpu')
# model1 = model1.to(device)



# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(lr = 1e-4, params = model1.parameters())
# optimizer.zero_grad()
# for epoch in range(10):
#   X_batches, y_batches = shuffle_and_batch(X_train_representations, y_train, 4, 67)
#   # X_batches = X_batches.to(device)
#   # y_batches = y_batches.to(device)
#   print('Epoch {}'.format(epoch))
#   for i in range(1):#len(X_batches)):
#     pred = model1(X_batches[i])
#     loss = criterion(pred,y_batches[i].long())
#     # print('Training Loss: {}'.format(loss))
#     loss.backward(retain_graph = True)
#     optimizer.step()


#   # Checking model on training set
#   train_preds = model1(X_train_representations)
#   train_loss = criterion(train_preds, y_train.long())
#   train_preds = labelize(train_preds)
#   train_prediction_comparisons = (y_train == train_preds)
#   train_accuracy = float(train_prediction_comparisons.sum())/float(y_train.shape[0])
#   print('Training Loss:{}\tTraining Accuracy:{}'.format(train_loss.item(), train_accuracy), end='  ')
  
#   # Checking model on testing set
#   test_preds = model1(X_test_representations)
#   test_loss = criterion(test_preds, y_test.long())
#   test_preds = labelize(test_preds)
#   test_prediction_comparisons = (y_test == test_preds)
#   test_accuracy = float(test_prediction_comparisons.sum())/float(y_test.shape[0])
#   print('Testing Loss:{}\tTesting Accuracy:{}'.format(test_loss.item(), test_accuracy))


In [ ]:
# torch.cuda.empty_cache()

In [ ]:
# if torch.cuda.is_available():
  # device = torch.device('cuda:0')
  # print('{} GPU(s)'.format(torch.cuda.device_count()))
# else:
  # device = torch.device('cpu')
  # print('CPU')

In [ ]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()

In [ ]:
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

# torch.cuda.empty_cache()
# printm()

In [ ]:
16280/1024

In [ ]:
# torch.cuda.is_available()

In [ ]:
# device = torch.device('cuda:0')
# device, torch.cuda.device_count()

In [ ]:
# xxxx = torch.from_numpy(X)
# XXXX = xxxx[:4]
# XXXX = XXXX.permute(0,3,1,2)

# flattened_channels = XXXX.reshape(3,-1)
# channel_mean = flattened_channels.mean(dim = 1)
# channel_stddev = flattened_channels.std(dim = 1)
# pppp = transforms.Compose([
                          # transforms.Normalize(channel_mean, channel_stddev)
# ])



In [ ]:
############################################################

In [ ]:
# import torch.multiprocessing as mp
# mp.cpu_count()

In [ ]:
###########################################################

In [ ]:
# class PCBdataset(Dataset):
#   def __init__(self, numpy_file, label_file, transform):
#     self.image_file = numpy_file
#     self.label_file = label_file
#     self.transform = transform
    
    
#   def __len__(self):
#     return self.image_file.shape[0]

#   def __getitem__(self, idx):

#     image = self.image_file[idx]
#     if(self.transform):
#       image = self.transform(self.image_file[idx])

#     di = {'image':image, 'label':self.label_file[idx].item()}
#     return di


# .permute(1,2,0)

In [ ]:
# import torchvision.transforms.functional as F
# t = F.to_pil_image(a[0]['image'])

In [ ]:
# a = PCBdata[0]['image']

In [ ]:
# plt.imshow(a)

In [ ]:
# a = torch.tensor([1.,2.,3.,4.])

# b = a[0]
# c = a[1]
# d = a[2]
# e = a[3]